In [4]:
!echo $ALPACA_API_ENDPOINT_PAPER

https://paper-api.alpaca.markets


In [5]:
import os

BASE_URL = 'https://api.alpaca.markets'
PAPER_URL = 'https://paper-api.alpaca.markets'

API_KEY = os.environ['ALPACA_API_KEY']
API_SECRET = os.environ['ALPACA_API_SECRET']

In [6]:
key = API_KEY
secret = API_SECRET

In [235]:
from alpaca.data.historical import CryptoHistoricalDataClient, StockHistoricalDataClient
from alpaca.trading import TradingClient
from alpaca.data.requests import CryptoBarsRequest, StockBarsRequest

from alpaca.trading.enums import AssetExchange, AssetClass, AccountStatus, AssetStatus

from datetime import datetime
from alpaca.data.timeframe import TimeFrame, TimeFrameUnit
import re
from enum import Enum

import pandas as pd

# set up clients
crypto_client = CryptoHistoricalDataClient()
stock_client = StockHistoricalDataClient(key, secret)
trading_client = TradingClient(key, secret)


In [273]:
start = '2022-09-01'
end = '2022-09-09'
timeframe = '20Min'

In [274]:
start = parse_date(start)
end = parse_date(end)
timeframe = parse_timeframe(timeframe)


In [102]:
symbol = ['BTC/USD']

In [262]:
symbol = ['AAPL', 'MMM']

In [263]:
def parse_timeframe(time_frame: str):

    match = re.search(r'(\d+)(\w+)', time_frame)

    if match:
        
        amount = int(match.group(1))
        unit = match.group(2)

        map = {
            'Min':TimeFrameUnit.Minute,
            'Hour':TimeFrameUnit.Hour,
            'Day': TimeFrameUnit.Day,
            'Week': TimeFrameUnit.Week,
            'Month': TimeFrameUnit.Month
        }

        return TimeFrame(amount, map[unit])
    else:
        raise ValueError(
            "Invalid timeframe format. Valid examples: 5Min, 3Hour, 10Day")

In [264]:
def parse_date(date: str):

    date_format = "%Y-%m-%d"
    dt = datetime.strptime(date, date_format)
    
    return dt

In [265]:
def get_stock_bars(symbol, timeframe, start, end):

    stock_request = StockBarsRequest(
        symbol_or_symbols=symbol, timeframe=timeframe, start=start, end=end)

    stock_bars = stock_client.get_stock_bars(stock_request).df

    return stock_bars


In [266]:
def get_crypto_bars(symbol, timeframe, start, end):

    crypto_request = CryptoBarsRequest(
        symbol_or_symbols=symbol, timeframe = timeframe, start=start, end=end)
    
    crypto_bars = crypto_client.get_crypto_bars(crypto_request).df
    
    return crypto_bars

In [108]:
crypto_bars = get_crypto_bars(symbol, timeframe, start, end)

APIError: {"message":"Invalid symbol(s): AAPL does not match ^[A-Z]+/[A-Z]+$"}


In [109]:
crypto_bars.df

AttributeError: 'DataFrame' object has no attribute 'df'

In [278]:
symbol = 'AAPL'

In [279]:
stock_bars = get_stock_bars(symbol, TimeFrame.Day, start, end)

In [280]:
stock_bars

open     high      low   close  \
symbol timestamp                                                      
AAPL   2022-09-01 04:00:00+00:00  156.640  158.420  154.670  157.96   
       2022-09-02 04:00:00+00:00  159.750  160.362  154.965  155.81   
       2022-09-06 04:00:00+00:00  156.470  157.090  153.690  154.53   
       2022-09-07 04:00:00+00:00  154.825  156.670  153.610  155.96   
       2022-09-08 04:00:00+00:00  154.640  156.360  152.680  154.46   

                                      volume  trade_count        vwap  
symbol timestamp                                                       
AAPL   2022-09-01 04:00:00+00:00  74224560.0     654608.0  156.523257  
       2022-09-02 04:00:00+00:00  77074411.0     646141.0  157.651625  
       2022-09-06 04:00:00+00:00  73679935.0     687134.0  155.011928  
       2022-09-07 04:00:00+00:00  87554883.0     696544.0  155.304640  
       2022-09-08 04:00:00+00:00  84907744.0     696902.0  154.394609

In [118]:
stock_bars.groupby('symbol').groups


{'AAPL': [('AAPL', 2022-09-01 06:00:00+00:00), ('AAPL', 2022-09-01 06:20:00+00:00), ('AAPL', 2022-09-01 07:00:00+00:00), ('AAPL', 2022-09-01 08:00:00+00:00), ('AAPL', 2022-09-01 08:20:00+00:00), ('AAPL', 2022-09-01 08:40:00+00:00), ('AAPL', 2022-09-01 09:00:00+00:00), ('AAPL', 2022-09-01 09:20:00+00:00), ('AAPL', 2022-09-01 09:40:00+00:00), ('AAPL', 2022-09-01 10:00:00+00:00), ('AAPL', 2022-09-01 10:20:00+00:00), ('AAPL', 2022-09-01 10:40:00+00:00), ('AAPL', 2022-09-01 11:00:00+00:00), ('AAPL', 2022-09-01 11:20:00+00:00), ('AAPL', 2022-09-01 11:40:00+00:00), ('AAPL', 2022-09-01 12:00:00+00:00), ('AAPL', 2022-09-01 12:20:00+00:00), ('AAPL', 2022-09-01 12:40:00+00:00), ('AAPL', 2022-09-01 13:00:00+00:00), ('AAPL', 2022-09-01 13:20:00+00:00), ('AAPL', 2022-09-01 13:40:00+00:00), ('AAPL', 2022-09-01 14:00:00+00:00), ('AAPL', 2022-09-01 14:20:00+00:00), ('AAPL', 2022-09-01 14:40:00+00:00), ('AAPL', 2022-09-01 15:00:00+00:00), ('AAPL', 2022-09-01 15:20:00+00:00), ('AAPL', 2022-09-01 15:40:00

In [130]:
groups = stock_bars.groupby('symbol')
groups.size()


symbol
AAPL    258
MMM     165
dtype: int64

In [129]:
for group in groups:
    print(group[1])

                                    open    high     low   close   volume  \
symbol timestamp                                                            
AAPL   2022-09-01 06:00:00+00:00  155.49  155.49  155.33  155.33    709.0   
       2022-09-01 06:20:00+00:00  155.29  155.29  155.29  155.29    207.0   
       2022-09-01 07:00:00+00:00  155.58  155.58  155.58  155.58    258.0   
       2022-09-01 08:00:00+00:00  156.00  156.42  156.00  156.34  34929.0   
       2022-09-01 08:20:00+00:00  156.24  156.39  155.88  156.00  27107.0   
...                                  ...     ...     ...     ...      ...   
       2022-09-08 22:20:00+00:00  154.32  154.34  154.26  154.30  19192.0   
       2022-09-08 22:40:00+00:00  154.35  154.40  154.31  154.40   8685.0   
       2022-09-08 23:00:00+00:00  154.38  154.56  154.34  154.50  24653.0   
       2022-09-08 23:20:00+00:00  154.50  154.57  154.30  154.50  21712.0   
       2022-09-08 23:40:00+00:00  154.56  154.69  154.52  154.60  28115.0   

In [184]:
def enum_to_val(item):
    if isinstance(item, Enum):
        return item.value
    return item

In [192]:
def enum_dict_to_val_dict(dict_):
    for key, val in dict_.items():
        dict_[key] = enum_to_val(val)
    return dict_

In [193]:
def get_all_positions():
    positions = trading_client.get_all_positions()
    positions_df = pd.DataFrame(enum_dict_to_val_dict(
        position.__dict__) for position in positions)
    return positions_df

In [196]:
positions = get_all_positions()

In [215]:
asset_classes = [member.name for member in AssetClass]


In [216]:
asset_classes

['US_EQUITY', 'CRYPTO']

In [218]:
exchanges = [member.name for member in AssetExchange]

In [219]:
exchanges

['AMEX',
 'ARCA',
 'BATS',
 'NYSE',
 'NASDAQ',
 'NYSEARCA',
 'FTXU',
 'CBSE',
 'GNSS',
 'ERSX',
 'OTC']

In [240]:
assets = trading_client.get_all_assets()

In [241]:
len(assets)

31491

In [242]:
assets = [asset for asset in assets if
          asset.status == AssetStatus.ACTIVE and
          asset.tradable
]

In [243]:
len(assets)

11069

In [239]:
asset

{   'asset_class': <AssetClass.US_EQUITY: 'us_equity'>,
    'easy_to_borrow': False,
    'exchange': <AssetExchange.OTC: 'OTC'>,
    'fractionable': False,
    'id': UUID('7d92777b-37fd-4fad-8e83-d9de3dde1ba9'),
    'maintenance_margin_requirement': 100.0,
    'marginable': False,
    'min_order_size': None,
    'min_trade_increment': None,
    'name': 'Peoples Insurance Company Group China Ltd Unsponsored ADR (China)',
    'price_increment': None,
    'shortable': False,
    'status': <AssetStatus.INACTIVE: 'inactive'>,
    'symbol': 'PINXY',
    'tradable': False}

In [221]:
asset.asset_class

<AssetClass.US_EQUITY: 'us_equity'>

In [222]:
asset.symbol

'PINXY'

In [223]:
asset.__dict__

{'id': UUID('7d92777b-37fd-4fad-8e83-d9de3dde1ba9'),
 'asset_class': <AssetClass.US_EQUITY: 'us_equity'>,
 'exchange': <AssetExchange.OTC: 'OTC'>,
 'symbol': 'PINXY',
 'name': 'Peoples Insurance Company Group China Ltd Unsponsored ADR (China)',
 'status': <AssetStatus.INACTIVE: 'inactive'>,
 'tradable': False,
 'marginable': False,
 'shortable': False,
 'easy_to_borrow': False,
 'fractionable': False,
 'min_order_size': None,
 'min_trade_increment': None,
 'price_increment': None,
 'maintenance_margin_requirement': 100.0}

In [229]:
account = trading_client.get_account()

In [231]:
account.status == AccountStatus.ACTIVE

True

In [226]:
account.cash

'-5363.47'

In [247]:
!pip install exchange_calendars

  Using cached exchange_calendars-4.2.5-py3-none-any.whl (189 kB)


In [258]:
import pandas as pd
import pandas_market_calendars as mcal

# Define the NYSE and NASDAQ calendars
calendar = mcal.get_calendar('NYSE')
schedule = calendar.schedule(start_date=start, end_date=end)


In [260]:
schedule

,market_open,market_close
2022-09-01,2022-09-01 13:30:00+00:00,2022-09-01 20:00:00+00:00
2022-09-02,2022-09-02 13:30:00+00:00,2022-09-02 20:00:00+00:00
2022-09-06,2022-09-06 13:30:00+00:00,2022-09-06 20:00:00+00:00
2022-09-07,2022-09-07 13:30:00+00:00,2022-09-07 20:00:00+00:00
2022-09-08,2022-09-08 13:30:00+00:00,2022-09-08 20:00:00+00:00
2022-09-09,2022-09-09 13:30:00+00:00,2022-09-09 20:00:00+00:00


In [9]:
n_rows = 1000_000
n = 6

In [10]:
step, remainder = divmod(n_rows, n)

In [11]:
for i in range(n):

    start = i * step + min(i, remainder)
    end = (i + 1) * step + min(i+1, remainder)
    print(start, end)


0 166667
166667 333334
333334 500001
500001 666668
666668 833334
833334 1000000


In [13]:
iterable = iter(range(9))
def func():
    x = next(iterable)
    yield x


In [17]:
next(func())

2